In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/song-data/songdata.csv
/kaggle/input/songs-recommendation-dataset/songdata.csv


# link of dataset 
https://www.kaggle.com/datasets/sraj39273/song-data

In [2]:
import pandas as pd
import numpy as np

In [3]:
df=pd.read_csv('/kaggle/input/song-data/songdata.csv')
df.head(3)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...


In [4]:
df.shape

(57650, 4)

In [5]:
df.dtypes

artist    object
song      object
link      object
text      object
dtype: object

In [6]:
df=df.sample(n=5000).drop('link',axis=1).reset_index(drop=True)

In [7]:
df.head(3)

,artist,song,text
0,Rolling Stones,Brand New Car,I got a brand new car \nAnd I like to drive r...
1,Raffi,To Everyone In All The World,"To everyone in all the world, \nI reach my ha..."
2,Kelly Family,I'll Swim I'll Swim,All the animals in the world \nWho's going to...


In [8]:
df.shape

(5000, 3)

In [9]:
df['text'] = df['text'].str.lower().replace(r'[^\w\s]','').replace(r'\n',' ', regex=True)

In [10]:
df['text'][0]

"i got a brand new car   and i like to drive real hard   i got a brand new car   and i'm feeling good so far      take her on the highway for a little spin   i want to see the kind of shape she's in   and i got a brand new star   jack her up baby, go on, open the hood   i want to check if her oil smells good   mmmm, smells like caviar      give her some stick   push her too far   right to the brink      hear the motor running   yeah, she's right in sink   tell me if she's thirsty   would she like a drink   and i'm stopping at this bar      slinky like a panther   you can hear her purr   touch her on the seat   go on, feel the fur   and i got a brand new star      feel the juice   foot to the floor   take some abuse      i got a brand new car   and i drive her in the dark   and i got a brand new car   i think i'll stop and park  "

In [11]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer=PorterStemmer()

def tokenization(txt):
    tokens=nltk.word_tokenize(txt)
    stemming=[stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [12]:
df['text']=df['text'].apply(lambda x:tokenization(x))

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [14]:
tfidvector=TfidfVectorizer(analyzer='word',stop_words='english')
matrix=tfidvector.fit_transform(df['text'])
similarity=cosine_similarity(matrix)

In [15]:
similarity[0]

array([1.        , 0.00576546, 0.05852052, ..., 0.03655948, 0.06756457,
       0.08344585])

In [16]:
df.loc[0]

artist                                       Rolling Stones
song                                          Brand New Car
text      i got a brand new car and i like to drive real...
Name: 0, dtype: object

In [17]:
df[df['song'] == 'Leave Me In Hell'].loc[0]

KeyError: 0

In [ ]:
df['song'].sample(4)

# Recommedation system

In [ ]:
def recommendation(song_df):
    # Check if song_df exists in the DataFrame
    if song_df not in df['song'].values:
        return "Song not found in the database"  # or return an empty list, depending on your use case
    
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])), reverse=True, key=lambda x: x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [ ]:
recommendation('Falling Down')

In [ ]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))